In [1]:
import numpy as np
from IPython.display import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
Image(url='nba-karl-malone.gif')

In [2]:
player_totals=pd.read_csv('./data/Player Totals.csv')
player_season_info=pd.read_csv('./data/Player Season Info.csv')
player_career_info=pd.read_csv('./data/Player Career Info.csv')
player_award_shares=pd.read_csv('./data/Player Award Shares.csv')
all_star=pd.read_csv('./data/All-Star Selections.csv')
advanced=pd.read_csv('./data/Advanced.csv')
end_of_season=pd.read_csv('./data/End of Season Teams.csv')

hof=player_career_info.drop(['birth_year','num_seasons'],axis=1)
player_names_and_seasons=hof[['player','first_seas','last_seas','player_id']]

In [3]:
seasons = np.flip(player_totals.season.unique())
seasons_leaderboards = {}
for season in seasons:
    season_top10s = {'pts': player_totals[player_totals['season'] == season].sort_values(by=['pts']).tail(10),
                     'reb': player_totals[player_totals['season'] == season].sort_values(by=['trb']).tail(10),
                     'ast': player_totals[player_totals['season'] == season].sort_values(by=['ast']).tail(10),
                     'min': player_totals[player_totals['season'] == season].sort_values(by=['mp']).tail(10),
                     'stl': player_totals[player_totals['season'] == season].sort_values(by=['stl']).tail(10),
                     'blk': player_totals[player_totals['season'] == season].sort_values(by=['blk']).tail(10)}
    seasons_leaderboards[season] = season_top10s

In [ ]:
leaderboard_points=[]
for i in range(len(player_names_and_seasons)):
    x=0
    if i%250==0:
        print(i)
    for season in range(player_names_and_seasons.iloc[i].first_seas,player_names_and_seasons.iloc[i].last_seas+1):
        top10_pts = seasons_leaderboards[season]['pts']
        top10_reb = seasons_leaderboards[season]['reb']
        top10_ast = seasons_leaderboards[season]['ast']
        top10_min = seasons_leaderboards[season]['min']
        top10_stl = seasons_leaderboards[season]['stl']
        top10_blk = seasons_leaderboards[season]['blk']
        for j in range(9,-1,-1):
            if top10_pts.iloc[j].player_id == player_names_and_seasons.iloc[i].player_id:
                x+=(5 + j/2)
            if top10_reb.iloc[j].player_id == player_names_and_seasons.iloc[i].player_id:
                x+=(5 + j/2)
            if top10_ast.iloc[j].player_id == player_names_and_seasons.iloc[i].player_id:
                x+=(5 + j/2)
            if top10_min.iloc[j].player_id == player_names_and_seasons.iloc[i].player_id:
                x+=(5 + j/2)
            if top10_stl.iloc[j].player_id == player_names_and_seasons.iloc[i].player_id:
                x+=(5 + j/2)
            if top10_blk.iloc[j].player_id == player_names_and_seasons.iloc[i].player_id:
                x+=(5 + j/2)
    leaderboard_points.append(x)
    
hof['Leaderboard points']=leaderboard_points        

0
250
500
750
1000
1250
1500
1750
2000
2250
2500
2750
3000
3250
3500
3750
4000


In [ ]:
allstar=[]
for index, player in player_names_and_seasons.iterrows():
    x=0
    all_stars=all_star[all_star.player==player.player]
    for index2, all_star in all_stars.iterrows():
        if all_star.season >= player.first_seas and all_star.season <= player.last_seas:
            x = x + 1
            
    allstar.append(x)

hof['All-Star appearances']=allstar        

In [ ]:
clean=pd.read_csv('./data/NBA_players_clean.csv')
clean.columns

In [ ]:
clean['Player']

In [ ]:
def cut_string(str):
    if '*' in str:
        return str.split('*')[0]
    else:
        return str
clean['player']=clean['Player'].apply(lambda x:cut_string(x))

In [ ]:
clean['player']

In [ ]:
rings_list=[]
for index, player in player_names_and_seasons.iterrows():
    x=0
    clean_players=clean[clean.player==player.player]
    for index2, clean_player in clean_players.iterrows():
        if clean_player.From == player.first_seas and clean_player.To <= player.last_seas:
            x = clean_player.Championships

    rings_list.append(x)    
            
hof['Championships']=rings_list     

In [ ]:
best5WSAverage=[]
for i in player_names_and_seasons.player_id:
    best5WSAverage.append(advanced[advanced['player_id']==i].nlargest(5, 'ws')['ws'].mean())
hof['Average of 5 highest WS seasons']=best5WSAverage


In [ ]:
best5BPMAverage=[]
for i in player_names_and_seasons.player_id:
    best5BPMAverage.append(advanced[advanced['player_id']==i].nlargest(5, 'bpm')['bpm'].mean())
hof['Average of 5 highest BPM seasons']=best5BPMAverage

In [ ]:
best5VORPAverage=[]
for i in player_names_and_seasons.player_id:
    best5VORPAverage.append(advanced[advanced['player_id']==i].nlargest(5, 'vorp')['vorp'].mean())
hof['Average of 5 highest VORP seasons']=best5VORPAverage

In [ ]:
heights=[]

for index, player in player_names_and_seasons.iterrows():
    x=clean.Height.mean()
    clean_players=clean[clean.player==player.player]
    for index2, clean_player in clean_players.iterrows():
        if clean_player.From == player.first_seas and clean_player.To <= player.last_seas:
            if not clean_player.Height.empty: 
                x = clean_player.Height
    
    heights.append(x)    
            
hof['Height']=heights


weights=[]

for index, player in player_names_and_seasons.iterrows():
    x=clean.Wt.mean()
    clean_players=clean[clean.player==player.player]
    for index2, clean_player in clean_players.iterrows():
        if clean_player.From == player.first_seas and clean_player.To <= player.last_seas:
            if not clean_player.Wt.empty: 
                x = clean_player.Wt
    
    weights.append(x)    
            
hof['Weight']=heights


In [ ]:
hof.isna().sum()

In [ ]:
hof['hof'] = hof['hof'].astype(int)

In [ ]:
hof['Average of 5 highest WS seasons'] = hof['Average of 5 highest WS seasons'].fillna(0)
hof['Average of 5 highest BPM seasons'] = hof['Average of 5 highest BPM seasons'].fillna(0)
hof['Average of 5 highest VORP seasons'] = hof['Average of 5 highest VORP seasons'].fillna(0)


In [ ]:
hof[hof['player']=='Patrick Ewing']

In [ ]:
hof.drop(3966,inplace=True)

In [ ]:
young=hof[(hof['last_seas']>2014) & (hof['hof']==0)]
old=hof[hof['last_seas']<=2014]
X=old.drop(['first_seas','last_seas','hof','player','player_id'],axis=1)
y=old['hof']
X_pred=young.drop(['first_seas','last_seas','hof','player','player_id'],axis=1)
y_pred=young['hof']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train,y_train)
pred_knn = knn.predict(X_test)
knn_prob=knn.predict_proba(X_pred)[:,-1]

logmodel=LogisticRegression(max_iter=10000)
logmodel.fit(X_train,y_train)
pred_log=logmodel.predict(X_test)
log_prob=logmodel.predict_proba(X_pred)[:,-1]

rfc=RandomForestClassifier(n_estimators=200)
rfc.fit(X_train,y_train)
pred_rfc=rfc.predict(X_test)
rfc_prob=rfc.predict_proba(X_pred)[:,-1]


print('Logistic Regression')
print(confusion_matrix(y_test,pred_log))
print(classification_report(y_test,pred_log))

print('Random Forest')
print(confusion_matrix(y_test,pred_rfc))
print(classification_report(y_test,pred_rfc))


print('K Nearest Neighbor')
print(confusion_matrix(y_test,pred_knn))
print(classification_report(y_test,pred_knn))




In [ ]:
res=pd.concat([X_pred,y_pred],axis=1)

#res['log_pred']=pred_log
res['log_prob']=log_prob

#res['rfc_pred']=pred_rfc
res['rfc_prob']=rfc_prob

#res['knn_pred']=pred_knn
res['knn_prob']=knn_prob

res_final=pd.concat([res,hof['player']],axis=1)

In [ ]:
final = res_final[['player','log_prob']]
final.sort_values(by=['log_prob'],ascending=False)

In [ ]:
pd.DataFrame(logmodel.coef_, columns=X_train.columns)

In [ ]:
logmodel.score(X_train,y_train)